## Setup imports

In [9]:
#!/usr/bin/env python3

import sevenbridges as sbg
import sys
import pdb
import copy
#from requests import request

#token = sys.argv[2]
config = sbg.Config(profile='cavatica')
api = sbg.Api(config=config)

## Get cram name using bs ids

In [10]:
project = 'kids-first-drc/sd-bhjxbdqk-har'
bs_list = open('/Users/brownm28/Documents/2018-Sep-10_proteomics_update/other_nantomics_file_update/matched_norm_bs_ids.txt')

for bs_id in bs_list:
    bs_id = bs_id.rstrip('\n')
    related = api.files.query(project=project, metadata = {'Kids First Biospecimen ID': bs_id})
    for f_obj in related:
        fn = f_obj.name
        parts = fn.split('.')
        if parts[-1] == 'cram':
            print (fn)

e252616f-bb41-48c0-b6c5-b01733dd4829.cram
e252616f-bb41-48c0-b6c5-b01733dd4829.cram.crai
f64f46a7-2d4a-4b93-b8ed-91cb11cd54f9.cram
f64f46a7-2d4a-4b93-b8ed-91cb11cd54f9.cram.crai
8f841d87-93ed-4339-b6ca-b55d96ce6adb.cram
8f841d87-93ed-4339-b6ca-b55d96ce6adb.cram.crai
24b7bdb6-9ab9-44d0-909c-e6d1271f1de1.cram
24b7bdb6-9ab9-44d0-909c-e6d1271f1de1.cram.crai
878c8533-24ae-4088-90db-d4e7949592bd.cram
878c8533-24ae-4088-90db-d4e7949592bd.cram.crai
3e52b97d-287b-49e9-b91f-be0f57358a9b.cram
3e52b97d-287b-49e9-b91f-be0f57358a9b.cram.crai
4622513a-c1fc-4027-a603-5e4af572c856.cram
4622513a-c1fc-4027-a603-5e4af572c856.cram.crai
d2a1e810-f420-47e9-87fb-d63c77fa61e8.cram
d2a1e810-f420-47e9-87fb-d63c77fa61e8.cram.crai
ceea70c8-a5ea-4ac3-b182-eae81566c1b7.cram
ceea70c8-a5ea-4ac3-b182-eae81566c1b7.cram.crai
adf5af38-0366-481c-9e26-4e11f47be50e.cram
adf5af38-0366-481c-9e26-4e11f47be50e.cram.crai
cec39440-5679-42f2-ba9d-188080fb8d48.cram
cec39440-5679-42f2-ba9d-188080fb8d48.cram.crai
ebfa8274-a152-48ab-9f

### Get DNA Info From Tasks

In [58]:
def find_dna_info(dna_proj, dna_list, api):
    dna_tasks = api.tasks.query(project=dna_proj).all()
    with open(dna_list) as f:
        dna_bs_ids = f.read()
    # pdb.set_trace()
    dna_t_names = {}

    for bs_id in dna_bs_ids.split('\n'):
        dna_t_names['alignment-' + bs_id] = 0
    for task in dna_tasks:
        if task.name in dna_t_names:
            try:
                fname = task.outputs['cram'].name
                dna_t_names[task.name] = 1
                print('\t'.join((task.name, task.id, fname)))
            except:
                sys.stderr.write(task.name + '\t' + task.status + '\n')
    for task in dna_t_names:
        if dna_t_names[task] == 0:
            sys.stderr.write('Missing\t' + task + '\n')

### Get RNA info from Tasks

In [36]:
def find_rna_info(rna_proj, rna_list, api):
    rna_tasks = api.tasks.query(project=rna_proj).all()
    with open(rna_list) as f:
        rna_t_names = f.read()
    for task in rna_tasks:
        if task.name in rna_t_names:
            fname = task.inputs['readFilesIn'][0].name.split('.')
            info = task.name.split(' ')
            print('\t'.join((task.name, task.id, fname[0], info[1])))
    

### copy files between projects

In [115]:
def find_copy_file(cur_project, source_project, fname, flag, suffix):
   
    try:
        fname = api.files.query(project=cur_project, names=[fname])[0]
        # return cur_cram
    except:
        sys.stderr.write(fname + ' not in' + cur_project + ', copying from ' + source_project + '\n')
        fname2 = fname + suffix

        try:
            source_fname = api.files.query(project=source_project, names=[fname])[0]
            source_fname.copy(project=cur_project, name=fname)
            if flag == 1:
                source_fname2 = api.files.query(project=source_project, names=[fname2])[0]
                source_fname2.copy(project=cur_project, name=fname2)
        except:
            sys.stderr.write('Cannot find file ' + fname + '\n')
            # return 'missing'
        # cur_fname = api.files.query(project=cur_project, names=[fname])[0]
        # return cur_cram

### Delete a metadata key

In [50]:
def delete_metadata_key(project, fname, key_list, api, f):
    flist = []
    flist.append(fname)
    if f == 1:
        flist.append(fname + '.crai')
    f_obj = api.files.query(project=project, names=flist)
    for obj in f_obj:
        cur = {}
        for key in obj.metadata:
            if key not in key_list:
                cur[key] = obj.metadata[key]
        obj.metadata = {}
        obj.metadata = cur
        obj.save()

### metadata update function

In [11]:
def update_meta(project, fname, meta_dict, api, f):
    flist = []
    flist.append(fname)
    if f == 1:
        flist.append(fname + '.crai')
    f_obj = api.files.query(project=project, names=flist)
    for obj in f_obj:
        for key in meta_dict:
            obj.metadata[key] = meta_dict[key]
            print (obj.name + '\t' + key + '\t' +  meta_dict[key])
        #obj.save()

## Query RNA

In [37]:
rna_proj = 'kfdrc-harmonization/sd-bhjxbdqk-06'
rna_list = '/Users/brownm28/Documents/2018-Sep-10_proteomics_update/query_inputs/rnaseq_task_names.txt'

rna_info = find_rna_info(rna_proj, rna_list, api)

RNAseq-BS_X67GTEM6 7316-1886	5ac55d70-6489-4731-bc9d-ccf946d833a0	723fc88f-132c-4ae5-ab16-23f8cd7951c2	7316-1886
RNAseq-BS_F4E8R9VY 7316-914	819f7898-3e4f-4749-becd-1e2ffbd9197f	c23dff00-99e5-4f39-81fe-51e0c2876157	7316-914


## Query DNA

In [59]:
tum_proj = 'kfdrc-harmonization/sd-bhjxbdqk-02'
tum_list = '/Users/brownm28/Documents/2018-Sep-10_proteomics_update/query_inputs/tumor_bsids.txt'
find_dna_info(tum_proj, tum_list, api)

alignment-BS_GXACEYKB	d07126f5-120d-431f-b396-637741770dba	d07126f5-120d-431f-b396-637741770dba.cram


alignment-BS_QMY84KF4	FAILED


alignment-BS_P3EVK3C2	c3f22937-b40c-4477-817e-592c1672b52b	c3f22937-b40c-4477-817e-592c1672b52b.cram
alignment-BS_GDPZN3SD	11f937ea-6562-4cf0-9983-c25051702ad8	11f937ea-6562-4cf0-9983-c25051702ad8.cram
alignment-BS_Z4Q8RH5M	7631868f-ba56-4dfa-a801-97838f9e3245	7631868f-ba56-4dfa-a801-97838f9e3245.cram
alignment-BS_JQNF0FQP	f35e0736-088c-4fa9-b111-4c14e698a720	f35e0736-088c-4fa9-b111-4c14e698a720.cram
alignment-BS_AG6M7TJE	cd3a4b45-81e0-47cb-b23b-92ef54a5e152	cd3a4b45-81e0-47cb-b23b-92ef54a5e152.cram
alignment-BS_RZNNEX1Y	59a2ab74-bd1d-48b4-8119-51971a1bce62	59a2ab74-bd1d-48b4-8119-51971a1bce62.cram
alignment-BS_8WJR4AA3	c2577280-05dc-4e28-a5c6-74b8f74e6a35	c2577280-05dc-4e28-a5c6-74b8f74e6a35.cram
alignment-BS_KYG3H6C6	d29137b5-c829-4e4e-9e21-e6cf89bac551	d29137b5-c829-4e4e-9e21-e6cf89bac551.cram
alignment-BS_THH0CE5A	3777448a-ef18-478f-b454-55fe0a01818a	3777448a-ef18-478f-b454-55fe0a01818a.cram
alignment-BS_A94A7EJ4	06175147-3479-41ad-a44f-398760208223	06175147-3479-41ad-a44f-39876020

Missing	alignment-BS_QMY84KF4
Missing	alignment-


## Update File Metadata

In [8]:
update_file = open('/Users/brownm28/Documents/2018-Sep-10_proteomics_update/other_nantomics_file_update/dna_meta_update.txt')
head = next(update_file)
tum_proj = 'kfdrc-harmonization/sd-bhjxbdqk-02'
norm_proj = 'kids-first-drc/sd-bhjxbdqk-har'
rna_proj = 'kfdrc-harmonization/sd-bhjxbdqk-06'

for line in update_file:
    data = line.rstrip('\n').split('\t')
    (n, t, r) = (9, -1, -1)
    #norm_cram = data[n]
    norm_cram = 'NULL'
    tum_cram = data[t]
    # tum_cram = 'NULL'
    # rna_bam = data[r]
    rna_bam = 'NULL'
    cur_meta = {}
    # cur_meta['sample_id'] = data[12]
    cur_meta['case_id'] = data[11]
    # there are 3 bs id columns, offset depends on working column of normal = 3, tumor = 4, rna = 5
    cur_meta['disease_type'] = data[-3]
    cur_meta['reference_genome'] = 'GRCh38'
    cur_meta['gender'] = data[12]
    cur_meta['ethnicity'] = data[13]
    cur_meta['race'] = data[14]
    if norm_cram != 'NULL':
        sys.stderr.write(norm_cram + '\t' + data[2] + '\n')
        cur_meta['sample_type'] = 'Normal'
        cur_meta['Kids First Biospecimen ID'] = data[2]
        cur_meta['experimental_strategy'] = 'WGS'
        update_meta(norm_proj, norm_cram, cur_meta, api, 1)
    if tum_cram != 'NULL':
        sys.stderr.write(tum_cram + '\t' + data[3] + '\n')
        cur_meta['Kids First Biospecimen ID'] = data[3]
        cur_meta['sample_type'] = 'Tumor'
        cur_meta['experimental_strategy'] = 'WGS'
        cur_meta['aliquot_id'] = 
        update_meta(tum_proj, tum_cram, cur_meta, api, 1)
    if rna_bam != 'NULL':
        sys.stderr.write(rna_bam + '\t' + data[6] + '\n')
        cur_meta['Kids First Biospecimen ID'] = data[6]
        cur_meta['sample_type'] = 'Tumor'
        cur_meta['experimental_strategy'] = 'RNA-seq'
        cur_meta['aliquot_id'] = data[8]
        update_meta(rna_proj, rna_bam, cur_meta, api, 0)

7316-242_750189.Aligned.toTranscriptome.out.bam	BS_W72364MN
7316-2502_746095.Aligned.toTranscriptome.out.bam	BS_FXHXQRKE


7316-242_750189.Aligned.toTranscriptome.out.bam	sample_id	7316-242
7316-242_750189.Aligned.toTranscriptome.out.bam	case_id	C752883
7316-242_750189.Aligned.toTranscriptome.out.bam	disease_type	Meningioma
7316-242_750189.Aligned.toTranscriptome.out.bam	reference_genome	GRCh38
7316-242_750189.Aligned.toTranscriptome.out.bam	gender	Male
7316-242_750189.Aligned.toTranscriptome.out.bam	ethnicity	Not Hispanic or Latino
7316-242_750189.Aligned.toTranscriptome.out.bam	race	White
7316-242_750189.Aligned.toTranscriptome.out.bam	Kids First Biospecimen ID	BS_W72364MN
7316-242_750189.Aligned.toTranscriptome.out.bam	sample_type	Tumor
7316-242_750189.Aligned.toTranscriptome.out.bam	experimental_strategy	RNA-seq
7316-242_750189.Aligned.toTranscriptome.out.bam	aliquot_id	750189
7316-2502_746095.Aligned.toTranscriptome.out.bam	sample_id	7316-2502
7316-2502_746095.Aligned.toTranscriptome.out.bam	case_id	C564078
7316-2502_746095.Aligned.toTranscriptome.out.bam	disease_type	Medulloblastoma
7316-2502_746095.

7316-2483_746086.Aligned.toTranscriptome.out.bam	BS_89M3B9GR
7316-2354_746083.Aligned.toTranscriptome.out.bam	BS_VSSDSYE8
7316-240_746085.Aligned.toTranscriptome.out.bam	BS_NDRDM2A9


7316-2354_746083.Aligned.toTranscriptome.out.bam	sample_id	7316-2354
7316-2354_746083.Aligned.toTranscriptome.out.bam	case_id	C562848
7316-2354_746083.Aligned.toTranscriptome.out.bam	disease_type	Low-grade glioma/astrocytoma (WHO grade I/II)
7316-2354_746083.Aligned.toTranscriptome.out.bam	reference_genome	GRCh38
7316-2354_746083.Aligned.toTranscriptome.out.bam	gender	Male
7316-2354_746083.Aligned.toTranscriptome.out.bam	ethnicity	Reported Unknown
7316-2354_746083.Aligned.toTranscriptome.out.bam	race	White
7316-2354_746083.Aligned.toTranscriptome.out.bam	Kids First Biospecimen ID	BS_VSSDSYE8
7316-2354_746083.Aligned.toTranscriptome.out.bam	sample_type	Tumor
7316-2354_746083.Aligned.toTranscriptome.out.bam	experimental_strategy	RNA-seq
7316-2354_746083.Aligned.toTranscriptome.out.bam	aliquot_id	746083
7316-240_746085.Aligned.toTranscriptome.out.bam	sample_id	7316-240
7316-240_746085.Aligned.toTranscriptome.out.bam	case_id	C55842
7316-240_746085.Aligned.toTranscriptome.out.bam	disease_ty

7316-2453_746094.Aligned.toTranscriptome.out.bam	BS_8MSZR1JN
7316-2534_746087.Aligned.toTranscriptome.out.bam	BS_85FRR415
7316-2389_746084.Aligned.toTranscriptome.out.bam	BS_0FRZD0VR


...

7316-261_750191.Aligned.toTranscriptome.out.bam	BS_593HEWBF
7316-1861_746092.Aligned.toTranscriptome.out.bam	BS_ZF8KKCKY
7316-1880_746091.Aligned.toTranscriptome.out.bam	BS_Y5CG43TJ


7316-261_750191.Aligned.toTranscriptome.out.bam	sample_id	7316-261
7316-261_750191.Aligned.toTranscriptome.out.bam	case_id	C48093
7316-261_750191.Aligned.toTranscriptome.out.bam	disease_type	Ganglioglioma
7316-261_750191.Aligned.toTranscriptome.out.bam	reference_genome	GRCh38
7316-261_750191.Aligned.toTranscriptome.out.bam	gender	Male
7316-261_750191.Aligned.toTranscriptome.out.bam	ethnicity	Not Hispanic or Latino
7316-261_750191.Aligned.toTranscriptome.out.bam	race	White
7316-261_750191.Aligned.toTranscriptome.out.bam	Kids First Biospecimen ID	BS_593HEWBF
7316-261_750191.Aligned.toTranscriptome.out.bam	sample_type	Tumor
7316-261_750191.Aligned.toTranscriptome.out.bam	experimental_strategy	RNA-seq
7316-261_750191.Aligned.toTranscriptome.out.bam	aliquot_id	750191
7316-1861_746092.Aligned.toTranscriptome.out.bam	sample_id	7316-1861
7316-1861_746092.Aligned.toTranscriptome.out.bam	case_id	C431976
7316-1861_746092.Aligned.toTranscriptome.out.bam	disease_type	Ganglioglioma
7316-1861_746092.

7316-554_746089.Aligned.toTranscriptome.out.bam	BS_MY9D8XZ1
7316-544_577731.Aligned.toTranscriptome.out.bam	BS_7GJT4A6A
7316-364_570114.Aligned.toTranscriptome.out.bam	BS_0448A413


7316-544_577731.Aligned.toTranscriptome.out.bam	sample_id	7316-544
7316-544_577731.Aligned.toTranscriptome.out.bam	case_id	C122877
7316-544_577731.Aligned.toTranscriptome.out.bam	disease_type	Brainstem glioma- Diffuse intrinsic pontine glioma
7316-544_577731.Aligned.toTranscriptome.out.bam	reference_genome	GRCh38
7316-544_577731.Aligned.toTranscriptome.out.bam	gender	Male
7316-544_577731.Aligned.toTranscriptome.out.bam	ethnicity	Not Hispanic or Latino
7316-544_577731.Aligned.toTranscriptome.out.bam	race	Other
7316-544_577731.Aligned.toTranscriptome.out.bam	Kids First Biospecimen ID	BS_7GJT4A6A
7316-544_577731.Aligned.toTranscriptome.out.bam	sample_type	Tumor
7316-544_577731.Aligned.toTranscriptome.out.bam	experimental_strategy	RNA-seq
7316-544_577731.Aligned.toTranscriptome.out.bam	aliquot_id	577731
7316-364_570114.Aligned.toTranscriptome.out.bam	sample_id	7316-364
7316-364_570114.Aligned.toTranscriptome.out.bam	case_id	C76137
7316-364_570114.Aligned.toTranscriptome.out.bam	disease_typ

BadRequest: The Disease type may only contain English letters, numbers, spaces, parentheses, and the symbols: '-', '_', '/', ';' and '.'.

## patch aliquot id

In [53]:
update_file = open('/Users/brownm28/Documents/2018-Sep-10_proteomics_update/add_aliquot_ids/metadata_table.txt')
head = next(update_file)
cheat_alias = 'cavatica/aysx-67gh'
tum_proj = cheat_alias
norm_proj = cheat_alias
rna_proj = cheat_alias

for line in update_file:
    data = line.rstrip('\n').split('\t')
    (n, t, r) = (9, 10, 11)
    norm_cram = data[n]
    tum_cram = data[t]
    rna_bam = data[r]
    cur_meta = {}
    if norm_cram != 'NULL':
        sys.stderr.write(norm_cram + '\t' + data[2] + '\n')
        cur_meta['aliquot_id'] = data[-3]
        update_meta(norm_proj, norm_cram, cur_meta, api, 1)
    if tum_cram != 'NULL':
        sys.stderr.write(tum_cram + '\t' + data[3] + '\n')
        cur_meta['aliquot_id'] = data[-2]
        update_meta(tum_proj, tum_cram, cur_meta, api, 1)
    if rna_bam != 'NULL':
        sys.stderr.write(rna_bam + '\t' + data[4] + '\n')
        cur_meta['aliquot_id'] = data[-1]
        update_meta(rna_proj, rna_bam, cur_meta, api, 0)

29997c0d-d2ef-4a1f-b631-727b338692b2.cram	BS_92HPNHFB


['_API', '__class__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_api', '_fields', '_name', '_parent', '_query', 'clear', 'copy', 'delete', 'equals', 'field', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'reload', 'setdefault', 'update', 'values']
29997c0d-d2ef-4a1f-b631-727b338692b2.cram	aliquot_id	550040
['_API', '__class__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', 

f35e0736-088c-4fa9-b111-4c14e698a720.cram	BS_JQNF0FQP
7316-120_571417.Aligned.toTranscriptome.out.bam	BS_B7X8VXQ3


['_API', '__class__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_api', '_fields', '_name', '_parent', '_query', 'clear', 'copy', 'delete', 'equals', 'field', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'reload', 'setdefault', 'update', 'values']
f35e0736-088c-4fa9-b111-4c14e698a720.cram.crai	aliquot_id	571315
['_API', '__class__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne

e252616f-bb41-48c0-b6c5-b01733dd4829.cram	BS_JQB0CTE7
7631868f-ba56-4dfa-a801-97838f9e3245.cram	BS_Z4Q8RH5M


...

d07126f5-120d-431f-b396-637741770dba.cram	BS_GXACEYKB
7316-914_571422.Aligned.toTranscriptome.out.bam	BS_F4E8R9VY


['_API', '__class__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_api', '_fields', '_name', '_parent', '_query', 'clear', 'copy', 'delete', 'equals', 'field', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'reload', 'setdefault', 'update', 'values']
d07126f5-120d-431f-b396-637741770dba.cram	aliquot_id	571328
['_API', '__class__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', 

ca1c0017-3ec7-48ac-9d46-ee225a1dad03.cram	BS_6M3B3TKJ
9a533419-b725-4747-93af-264d52e059ad.cram	BS_WB8P21ED


['_API', '__class__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__slotnames__', '__str__', '__subclasshook__', '__weakref__', '_api', '_fields', '_name', '_parent', '_query', 'clear', 'copy', 'delete', 'equals', 'field', 'fromkeys', 'get', 'items', 'keys', 'pop', 'popitem', 'reload', 'setdefault', 'update', 'values']
ca1c0017-3ec7-48ac-9d46-ee225a1dad03.cram	aliquot_id	549932
['_API', '__class__', '__contains__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', 

7316-957_571420.Aligned.toTranscriptome.out.bam	BS_T4JFN54G


## del key from metadata

In [51]:
update_file = open('/Users/brownm28/Documents/2018-Sep-10_proteomics_update/add_aliquot_ids/metadata_table.txt')
head = next(update_file)
tum_proj = 'kfdrc-harmonization/sd-bhjxbdqk-02'
norm_proj = 'kids-first-drc/sd-bhjxbdqk-har'
rna_proj = 'kfdrc-harmonization/sd-bhjxbdqk-06'
key_list = ['external_aliquot_id']
for line in update_file:
    data = line.rstrip('\n').split('\t')
    (n, t, r) = (9, 10, 11)
    norm_cram = data[n]
    tum_cram = data[t]
    rna_bam = data[r]
    cur_meta = {}
    if norm_cram != 'NULL':
        sys.stderr.write(norm_cram + '\t' + data[2] + '\n')
        delete_metadata_key(norm_proj, norm_cram, key_list, api, 1)
    if tum_cram != 'NULL':
        sys.stderr.write(tum_cram + '\t' + data[3] + '\n')
        delete_metadata_key(tum_proj, tum_cram, key_list, api, 1)
    if rna_bam != 'NULL':
        sys.stderr.write(rna_bam + '\t' + data[4] + '\n')
        delete_metadata_key(rna_proj, rna_bam, key_list, api, 0)

29997c0d-d2ef-4a1f-b631-727b338692b2.cram	BS_92HPNHFB
f35e0736-088c-4fa9-b111-4c14e698a720.cram	BS_JQNF0FQP
7316-120_571417.Aligned.toTranscriptome.out.bam	BS_B7X8VXQ3
e252616f-bb41-48c0-b6c5-b01733dd4829.cram	BS_JQB0CTE7
7631868f-ba56-4dfa-a801-97838f9e3245.cram	BS_Z4Q8RH5M
7316-121_571416.Aligned.toTranscriptome.out.bam	BS_66ABA973
edc0aa2e-ba54-40b0-85a1-e392e7a4e84f.cram	BS_W75GCHPW
06175147-3479-41ad-a44f-398760208223.cram	BS_A94A7EJ4
7316-1790_573148.Aligned.toTranscriptome.out.bam	BS_MV6ASWV9
8e91ab29-acff-4a68-941b-8171f3dcc4fb.cram	BS_S5SSWSVM
7316-1886_711732.Aligned.toTranscriptome.out.bam	BS_X67GTEM6
39a54746-b2b1-47de-8dcd-6db2648352fb.cram	BS_Q1SPJ467
c2577280-05dc-4e28-a5c6-74b8f74e6a35.cram	BS_8WJR4AA3
7316-2141_571446.Aligned.toTranscriptome.out.bam	BS_MMNRZ1EQ
805ca5a6-39bd-4ba0-ac03-983a81cde8be.cram	BS_XRF8D151
d29137b5-c829-4e4e-9e21-e6cf89bac551.cram	BS_KYG3H6C6
7316-2172_571451.Aligned.toTranscriptome.out.bam	BS_S50KQEXV
878c8533-24ae-4088-90db-d4e7949592bd.cram	

## Copy files

In [116]:
update_file = open('/Users/brownm28/Documents/2018-Sep-10_proteomics_update/file_metadata_update.txt')
head = next(update_file)
tum_proj = 'kfdrc-harmonization/sd-bhjxbdqk-02'
norm_proj = 'kids-first-drc/sd-bhjxbdqk-har'
rna_proj = 'kfdrc-harmonization/sd-bhjxbdqk-06'
dest_proj = 'cavatica/aysx-67gh'
for line in update_file:
    data = line.rstrip('\n').split('\t')
    (n, t, r) = (9, 10, 11)
    norm_cram = data[n]
    tum_cram = data[t]
    rna_bam = data[r]
    # find_copy_file(cur_project, source_project, fname, f, suffix)
    if norm_cram != 'NULL':
        sys.stderr.write(norm_cram + '\t' + data[2] + '\n')
        find_copy_file(dest_proj, norm_proj, norm_cram, 1, '.crai')
    if tum_cram != 'NULL':
        sys.stderr.write(tum_cram + '\t' + data[3] + '\n')
        find_copy_file(dest_proj, tum_proj, tum_cram, 1, '.crai')
    if rna_bam != 'NULL':
        sys.stderr.write(rna_bam + '\t' + data[4] + '\n')
        find_copy_file(dest_proj, rna_proj, rna_bam, 0, '')

29997c0d-d2ef-4a1f-b631-727b338692b2.cram	BS_92HPNHFB
f35e0736-088c-4fa9-b111-4c14e698a720.cram	BS_JQNF0FQP
7316-120_571417.Aligned.toTranscriptome.out.bam	BS_B7X8VXQ3
7316-120_571417.Aligned.toTranscriptome.out.bam not incavatica/aysx-67gh, copying from kfdrc-harmonization/sd-bhjxbdqk-06
e252616f-bb41-48c0-b6c5-b01733dd4829.cram	BS_JQB0CTE7
e252616f-bb41-48c0-b6c5-b01733dd4829.cram not incavatica/aysx-67gh, copying from kids-first-drc/sd-bhjxbdqk-har
7631868f-ba56-4dfa-a801-97838f9e3245.cram	BS_Z4Q8RH5M
7631868f-ba56-4dfa-a801-97838f9e3245.cram not incavatica/aysx-67gh, copying from kfdrc-harmonization/sd-bhjxbdqk-02
7316-121_571416.Aligned.toTranscriptome.out.bam	BS_66ABA973
7316-121_571416.Aligned.toTranscriptome.out.bam not incavatica/aysx-67gh, copying from kfdrc-harmonization/sd-bhjxbdqk-06
edc0aa2e-ba54-40b0-85a1-e392e7a4e84f.cram	BS_W75GCHPW
06175147-3479-41ad-a44f-398760208223.cram	BS_A94A7EJ4
06175147-3479-41ad-a44f-398760208223.cram not incavatica/aysx-67gh, copying from kfd